In [83]:
from pathlib import Path

import json
import pandas as pd

import preprocessing
from feature_engineering import FeatureCreator,PredictionFeatureCreator
import prediction
# from feature_engineering import PredictionFeatureCreator
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [96]:
%autoreload

In [20]:
#レース結果テーブルの前処理
results_preprocessed = preprocessing.process_results()

In [21]:
results_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 88363 entries, 0 to 88362
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   race_id      88363 non-null  int64  
 1   horse_id     88363 non-null  int64  
 2   jockey_id    88363 non-null  int64  
 3   trainer_id   88363 non-null  int64  
 4   rank         88363 non-null  int64  
 5   sex          88363 non-null  int64  
 6   age          88363 non-null  int64  
 7   weight       88363 non-null  int64  
 8   weight_diff  88363 non-null  int64  
 9   tansho_odds  88363 non-null  float64
 10  popularity   88363 non-null  int64  
 11  impost       88363 non-null  float64
 12  wakuban      88363 non-null  int64  
 13  umaban       88363 non-null  int64  
dtypes: float64(2), int64(12)
memory usage: 9.4 MB


In [22]:
#重複チェック
results_preprocessed.duplicated(subset=["race_id","horse_id"]).sum()

np.int64(0)

In [23]:
#欠損チェック
results_preprocessed.isnull().sum()

race_id        0
horse_id       0
jockey_id      0
trainer_id     0
rank           0
sex            0
age            0
weight         0
weight_diff    0
tansho_odds    0
popularity     0
impost         0
wakuban        0
umaban         0
dtype: int64

In [24]:
#馬の過去成績テーブルの加工
horse_results_preprocessed = preprocessing.process_horse_results()
horse_results_preprocessed

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
0,2016100158,2024-07-27,12,0.0,3.6,1.0,3,3250,0.0,NaN,13.0
1,2016100158,2024-06-22,10,0.0,4.1,0.0,3,3110,1.0,NaN,14.0
2,2016100158,2024-06-01,9,0.0,1.6,0.0,0,1800,1.0,5.0,10.0
3,2016100158,2024-04-13,7,0.0,4.3,0.0,3,4250,0.0,NaN,12.0
4,2016100158,2023-12-23,6,0.0,4.5,0.0,3,4100,0.0,NaN,12.0
...,...,...,...,...,...,...,...,...,...,...,...
221487,2021106341,2024-05-31,5,5.4,2.5,1.0,0,1500,3.0,NaN,12.0
221488,2021106341,2024-05-06,10,0.0,3.0,1.0,0,1500,0.0,NaN,12.0
221489,2021106341,2024-04-25,7,0.0,1.3,0.0,0,1500,2.0,NaN,12.0
221490,2021106341,2024-04-12,12,0.0,4.0,0.0,0,1500,0.0,NaN,12.0


In [33]:
horse_results_preprocessed.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221492 entries, 0 to 221491
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   horse_id      221492 non-null  int64         
 1   date          221492 non-null  datetime64[ns]
 2   rank          221492 non-null  int64         
 3   prize         221492 non-null  float64       
 4   rank_diff     221185 non-null  float64       
 5   weather       221010 non-null  float64       
 6   race_type     221492 non-null  int64         
 7   course_len    221492 non-null  int64         
 8   ground_state  221481 non-null  float64       
 9   race_class    152059 non-null  float64       
 10  n_horses      221485 non-null  float64       
dtypes: datetime64[ns](1), float64(6), int64(4)
memory usage: 18.6 MB


In [12]:
horse_id_list = [
    2020102562,
    2020104817
]
horse_results_preprocessed.query("horse_id in @horse_id_list")

,horse_id,date,rank,prize,rank_diff,weather,race_type,course_len,ground_state,race_class,n_horses
52500,2020102562,2024-10-19,11,0.0,1.6,0.0,0,1400,0.0,3.0,16.0
52501,2020102562,2024-08-18,15,0.0,4.2,0.0,0,1800,0.0,5.0,16.0
52502,2020102562,2024-05-19,13,0.0,1.8,1.0,0,1600,0.0,5.0,15.0
52503,2020102562,2024-05-04,12,0.0,1.6,0.0,0,1600,1.0,3.0,13.0
52504,2020102562,2024-01-28,1,800.0,0.0,1.0,0,1600,0.0,2.0,14.0
52505,2020102562,2023-12-23,9,0.0,0.9,0.0,0,1200,0.0,2.0,16.0
52506,2020102562,2023-10-08,6,0.0,0.5,1.0,0,1400,0.0,2.0,15.0
52507,2020102562,2023-09-18,3,200.0,0.4,0.0,0,1200,0.0,2.0,16.0
52508,2020102562,2023-08-27,4,120.0,0.3,0.0,0,1200,0.0,2.0,15.0
52509,2020102562,2023-05-06,1,550.0,0.0,0.0,0,1400,0.0,1.0,16.0


In [30]:
race_info_preprocessed = preprocessing.process_race_info()

In [31]:
race_info_preprocessed

,race_id,date,race_type,around,course_len,weather,ground_state,race_class,place
0,202301010101,2023-07-22,1,0.0,1200,0.0,0,1,1
1,202301010102,2023-07-22,0,0.0,1000,0.0,0,1,1
2,202301010103,2023-07-22,0,0.0,1700,0.0,0,1,1
3,202301010104,2023-07-22,1,0.0,1500,0.0,0,1,1
4,202301010105,2023-07-22,0,0.0,1700,0.0,0,0,1
...,...,...,...,...,...,...,...,...,...
6569,202410030808,2024-07-21,0,0.0,1000,0.0,1,2,10
6570,202410030809,2024-07-21,1,0.0,2000,1.0,0,5,10
6571,202410030810,2024-07-21,1,0.0,1200,1.0,0,3,10
6572,202410030811,2024-07-21,1,0.0,1800,0.0,0,5,10


In [27]:
race_info_preprocessed.isnull().sum()

race_id           0
date              0
race_type         0
around          290
course_len       57
weather           5
ground_state      0
race_class        0
place             0
dtype: int64

In [28]:
fc = FeatureCreator()
features = fc.create_features()

In [ ]:
# features.duplicated(subset=["race_id", "horse_id"]).sum()

np.int64(0)

In [ ]:
予測当日の処理

In [10]:
import preprocessing

horse_results_preprocessed = preprocessing.process_horse_results(
    save_filename="horse_results_prediction.csv"
)

In [2]:
%autoreload

UsageError: Line magic function `%autoreload` not found.


In [71]:
# 特徴量作成
pfc = PredictionFeatureCreator()
#馬の過去成績集計は事前に行っておいても良い
pfc.agg_horse_n_races()

Current working directory: c:\Users\USER\Downloads\Keiba-YosokuAI-main\Keiba-YosokuAI-main\KeibaAI-training\v1_0_0\src
Resolved file path: C:\Users\USER\Downloads\Keiba-YosokuAI-main\Keiba-YosokuAI-main\KeibaAI-training\common\data\prediction_population\population.csv
Does the file exist? True


In [72]:
pfc.agg_horse_n_races_df

,date,race_id,horse_id,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races
0,2024-12-22,202406050807,2019106972,7.666667,0.000000,9.6,0.00,11.200000,0.000000,6.709677,15.461290
1,2024-12-22,202406050808,2021100161,7.333333,260.000000,5.0,310.00,6.285714,221.428571,6.285714,221.428571
2,2024-12-22,202406050808,2019105064,8.666667,0.000000,6.2,206.00,5.500000,199.000000,4.684211,184.736842
3,2024-12-22,202406050809,2021105499,4.666667,323.333333,5.0,226.00,4.750000,237.500000,4.750000,237.500000
4,2024-12-22,202406050812,2020110136,6.666667,582.666667,5.6,475.64,7.100000,312.820000,6.733333,305.213333
5,2024-12-22,202408070807,2021102336,3.000000,213.333333,3.8,238.00,5.500000,119.000000,5.545455,108.181818
6,2024-12-22,202408070807,2018103388,5.666667,53.333333,4.8,112.00,5.200000,88.000000,5.000000,124.551724
7,2024-12-22,202408070808,2020102468,9.000000,266.666667,6.6,248.00,6.100000,164.000000,5.360000,125.080000
8,2024-12-22,202408070808,2020102687,9.333333,0.000000,7.0,160.00,5.400000,169.000000,4.666667,163.666667
9,2024-12-22,202408070810,2020101486,6.000000,652.966667,5.0,539.86,6.000000,433.720000,5.071429,455.685714


In [73]:
# 特徴量の更新
feature = pfc.create_features(
    race_id="202406050811",
    skip_agg_horse=True # 事前に集計した場合はスキップできる
)
# print(feature)

c:\Users\USER\Downloads\Keiba-YosokuAI-main\Keiba-YosokuAI-main\KeibaAI-training\v1_0_0\src\feature_engineering.py:220: FutureWarning: Passing literal html to 'read_html' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  df = pd.read_html(html)[0]


In [74]:
feature

,date,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,...,race_class,place,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races
0,2024-12-22,202406050811,2021105143,660,1164,1,1,4.0,2,56.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-12-22,202406050811,2019105283,666,1061,2,1,0.0,0,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-12-22,202406050811,2021105436,5339,1147,3,2,2.8,1,56.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-12-22,202406050811,2019105748,1179,1176,4,2,35.1,11,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2024-12-22,202406050811,2020102781,1140,1168,5,3,7.1,3,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,2024-12-22,202406050811,2019105552,5626,1162,6,3,17.2,7,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,2024-12-22,202406050811,2019104740,1088,1118,7,4,17.6,8,56.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,2024-12-22,202406050811,2021105898,5386,1126,8,4,10.9,4,54.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,2024-12-22,202406050811,2017102170,732,1058,9,5,49.7,12,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,2024-12-22,202406050811,2018104541,1122,1137,10,5,16.4,6,58.0,...,5,6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [59]:
race_id="202406050811"
pfc.htmls[race_id]

'<html><head><script type="text/javascript" async="" src="https://static.criteo.net/js/ld/publishertag.prebid.144.js"></script><script type="text/javascript" async="" src="https://script.4dex.io/localstore.js"></script><script async="" src="https://www.clarity.ms/s/0.7.59/clarity.js"></script><script type="text/javascript" async="" src="//vidstat.taboola.com/lite-unit/5.1.1/UnitSliderDesktop.min.js"></script><script type="text/javascript" src="https://cdn.taboola.com/scripts/eid.es5.js" crossorigin="anonymous" async="async"></script><script type="text/javascript" src="https://cdn.taboola.com/scripts/fraud-detect.js" crossorigin="anonymous" async="async"></script><script type="text/javascript" src="https://cdn.taboola.com/scripts/cds-pips.js" crossorigin="anonymous" async="async"></script><script type="text/javascript" src="https://gum.criteo.com/sync?c=72&amp;r=2&amp;j=TRC.getRTUS&amp;us_privacy=&amp;gdpr=0&amp;gdpr_consent=" async="async"></script><script type="text/javascript" async=

In [66]:
# features = (
#             pfc.population.merge(pfc.results, on=["race_id", "horse_id"])
#             .merge(pfc.race_info, on="race_id")
#             .merge(
#                 pfc.agg_horse_n_races_df,
#                 on=["race_id", "date", "horse_id"],
#                 how="left",
#             )
#         )
# print("Population race_id:", pfc.population["race_id"].unique())
# print("Results race_id:", pfc.results["race_id"].unique())
# print("Race Info race_id:", pfc.race_info["race_id"].unique())
# print("Agg Horse N Races race_id:", pfc.agg_horse_n_races_df["race_id"].unique())


Population race_id: [202405030101 202405030102 202405030103 202405030104 202405030105
 202405030106 202405030107 202405030108 202405030109 202405030110
 202405030111 202405030112 202408040101 202408040102 202408040103
 202408040104 202408040105 202408040106 202408040107 202408040108
 202408040109 202408040110 202408040111 202408040112]
Results race_id: [202406050811]
Race Info race_id: [202406050811]
Agg Horse N Races race_id: [202405030101 202405030102 202405030103 202405030104 202405030106
 202405030107 202405030108 202405030109 202405030110 202405030111
 202405030112 202408040101 202408040102 202408040103 202408040104
 202408040106 202408040107 202408040108 202408040109 202408040110
 202408040111 202408040112]


In [70]:
pfc.agg_horse_n_races_df

,date,race_id,horse_id,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races
0,2024-06-01,202405030101,2019100108,6.666667,66.666667,7.6,40.0,5.900000,208.000000,4.538462,218.500000
1,2024-06-01,202405030101,2019104899,8.666667,0.000000,10.2,0.0,9.300000,134.710000,6.764706,396.276471
2,2024-06-01,202405030101,2016103092,8.666667,0.000000,8.8,0.0,7.700000,12.000000,6.945946,8.500000
3,2024-06-01,202405030101,2020102800,6.666667,80.000000,8.0,48.0,6.900000,184.900000,6.461538,186.230769
4,2024-06-01,202405030101,2019105143,13.000000,0.000000,13.4,0.0,11.600000,158.640000,8.687500,193.525000
...,...,...,...,...,...,...,...,...,...,...,...
296,2024-06-01,202408040112,2019104288,2.000000,470.000000,3.2,306.0,3.000000,277.000000,5.681818,176.409091
297,2024-06-01,202408040112,2018104208,9.000000,76.666667,7.4,104.0,7.800000,63.400000,5.648649,145.200000
298,2024-06-01,202408040112,2020104845,8.666667,0.000000,7.8,0.0,6.500000,140.500000,6.615385,114.076923
299,2024-06-01,202408040112,2021100161,4.333333,443.333333,4.4,310.0,5.666667,258.333333,5.666667,258.333333


In [64]:
pfc.race_info

,race_id,race_type,around,course_len,weather,ground_state,race_class,place
0,202406050811,1,0,2500,0,0,5,6


In [63]:
pfc.results

,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,sex,age,weight,weight_diff
0,202406050811,2021105143,660,1164,1,1,4.0,2,56.0,0,3,520,-2
1,202406050811,2019105283,666,1061,2,1,0.0,0,58.0,0,5,0,0
2,202406050811,2021105436,5339,1147,3,2,2.8,1,56.0,0,3,516,6
3,202406050811,2019105748,1179,1176,4,2,35.1,11,58.0,0,5,432,2
4,202406050811,2020102781,1140,1168,5,3,7.1,3,58.0,0,4,512,-2
5,202406050811,2019105552,5626,1162,6,3,17.2,7,58.0,0,5,506,0
6,202406050811,2019104740,1088,1118,7,4,17.6,8,56.0,1,5,498,-2
7,202406050811,2021105898,5386,1126,8,4,10.9,4,54.0,1,3,468,-4
8,202406050811,2017102170,732,1058,9,5,49.7,12,58.0,0,7,502,-14
9,202406050811,2018104541,1122,1137,10,5,16.4,6,58.0,0,6,474,0


,date,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,...,race_class,place,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races


In [69]:
pfc.population

,date,race_id,horse_id
0,2024-06-01,202405030101,2019100108
1,2024-06-01,202405030101,2019104899
2,2024-06-01,202405030101,2016103092
3,2024-06-01,202405030101,2020102800
4,2024-06-01,202405030101,2019105143
...,...,...,...
323,2024-06-01,202408040112,2019104288
324,2024-06-01,202408040112,2018104208
325,2024-06-01,202408040112,2020104845
326,2024-06-01,202408040112,2021100161


In [55]:
pfc.race_info

,race_id,race_type,around,course_len,weather,ground_state,race_class,place
0,202406050811,1,0,2500,0,0,5,6


In [56]:
feature

,date,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,...,race_class,place,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races


In [98]:
# features = features.fillna(0)  # または適切な値で埋める


,date,race_id,horse_id,jockey_id,trainer_id,umaban,wakuban,tansho_odds,popularity,impost,...,race_class,place,rank_3races,prize_3races,rank_5races,prize_5races,rank_10races,prize_10races,rank_1000races,prize_1000races


In [105]:
prediction.predict(feature)

,race_id,umaban,tansho_odds,popularity,pred
14,202406050811,15,57.1,13,0.055882
15,202406050811,16,30.1,10,0.020471
8,202406050811,9,49.7,12,0.009400
11,202406050811,12,108.1,14,0.008815
9,202406050811,10,16.4,6,0.008746
1,202406050811,2,0.0,0,0.007424
13,202406050811,14,122.4,15,0.006880
0,202406050811,1,4.0,2,0.006013
10,202406050811,11,10.9,5,0.005580
3,202406050811,4,35.1,11,0.005161


In [104]:
%autoreload

In [91]:
print("Features shape:", features.shape)


Features shape: (0, 29)


In [94]:
print("Feature columns:", feature_cols)
print("Missing feature columns:", set(feature_cols) - set(features.columns))


NameError: name 'feature_cols' is not defined

In [89]:
feature.columns

Index(['date', 'race_id', 'horse_id', 'jockey_id', 'trainer_id', 'umaban',
       'wakuban', 'tansho_odds', 'popularity', 'impost', 'sex', 'age',
       'weight', 'weight_diff', 'race_type', 'around', 'course_len', 'weather',
       'ground_state', 'race_class', 'place', 'rank_3races', 'prize_3races',
       'rank_5races', 'prize_5races', 'rank_10races', 'prize_10races',
       'rank_1000races', 'prize_1000races'],
      dtype='object')